In [1]:
import os
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import numpy as np
import tqdm as notebook_tqdm
from torchvision import datasets
import torchvision
from torchvision.models.resnet import resnet18, resnet50
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime
import numpy as np 

from PIL import Image
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
import torchvision 
import torchvision.transforms as transforms 
from torchvision.transforms import ToTensor
from torchvision import datasets 
from datetime import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    torch.cuda.empty_cache()

/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
device

device(type='cuda')

# Normal training / test  loop 

In [24]:
class GTSRB(Dataset):
    """ Taken from https://github.com/tomlawrenceuk/GTSRB-Dataloader"""
    base_folder = 'GTSRB'

    def __init__(self, root_dir, train=False, transform=None):
        self.root_dir = root_dir
        self.sub_directory = 'trainingset' if train else 'testset'  # TODO: RENAME
        self.csv_file_name = 'training.csv' if train else 'test.csv'

        csv_file_path = os.path.join(root_dir, self.base_folder, self.sub_directory, self.csv_file_name)  # paths file
        self.csv_data = pd.read_csv(csv_file_path)
        self.transform = transform

        self.memorize_samples = [None] * len(self.csv_data)

    def __len__(self):
        return len(self.csv_data)

    def __getitem__(self, idx):
        # memorize:
        if self.memorize_samples[idx] is not None:
            return self.memorize_samples[idx]
        img_path = os.path.join(self.root_dir, self.base_folder, self.sub_directory, self.csv_data.iloc[idx, 0])
        img = Image.open(img_path)
        img_label = self.csv_data.iloc[idx, 1]

        if self.transform is not None:
            img = self.transform(img)

        self.memorize_samples[idx] = (img, img_label)
        return img, img_label


def get_data_labels(dataset: Dataset):
    """ split dataset into tensor of data inputs and their labels"""
    inputs = []
    targets = []
    for i in range(len(dataset)):
        x, y = dataset[i]
        inputs.append(x)
        targets.append(y)

    inputs = torch.stack(inputs)
    targets = torch.tensor(targets)
    return inputs, targets


def get_train_val_dls(dataset: Dataset, batch_size):
    """
    same as previous function without test (use this function in case that test is already separated).
    all inputs as in prev function (i.e. get_train_val_test_dls).
    :return: train & val data loaders of type DataLoader.
    """

    ds_size = len(dataset)
    indices = list(range(ds_size))
    np.random.shuffle(indices)
    val_split_idx = int(ds_size * 0.7)
    train_indices, val_indices = indices[:val_split_idx], indices[val_split_idx:]

    # train_subset = Subset(dataset, train_indices)
    # val_subset = Subset(dataset, val_indices)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=SubsetRandomSampler(train_indices))
    validation_loader = DataLoader(dataset, batch_size=batch_size, sampler=SubsetRandomSampler(val_indices))

    return train_loader, validation_loader


def dataset_to_dataloader(dataset: Dataset, batch_size):
    """ A function that gets dataset and returns simple data loader.
        It is useful when the dataset comes slitted (to train and test)"""

    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

data_root_dir = os.path.join(".", "data")

In [6]:
# defining hyperparameters  
epochs = 30   #the nn will train 31 times 
learning_rate = 0.0001 #how much the weight will be updated each time 
batch_size = 64 
classes = 43 
img_size = 32
random_seed = 42

In [7]:
from torchvision import transforms
transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.3403, 0.3121, 0.3214),
                             (0.2724, 0.2608, 0.2669))  # To get [0,1] range
    ])

In [8]:
def get_train_valid_loader(
                           batch_size,
                           augment,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True,
                           num_workers=2):

    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg


    # load the dataset

    base_dataset = datasets.ImageFolder(
        root='/volumes1/thesis/notebooks/data/gtsrb/GTSRB/Training', transform=transforms)

    # TODO
    split_datasets = torch.utils.data.random_split(base_dataset, [0.20,0.8])
    global val_dataset, train_dataset
    val_dataset = split_datasets[0]
    train_dataset = split_datasets[1]
    

    global num_train 
    num_train= len(train_dataset)
    indices = list(range(num_train))
    global split 
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)


    #train_idx, valid_idx = indices[split:], indices[:split]
    #train_sampler = SubsetRandomSampler(train_idx)
    #valid_sampler = SubsetRandomSampler(valid_idx)


    global train_loader 
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size,
        num_workers=num_workers, 
        #sampler = train_sampler
    )
    global valid_loader 
    valid_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size,
        num_workers=num_workers, 
        #sampler = valid_sampler
    )

    return train_loader, valid_loader

print(get_train_valid_loader(batch_size = 64, augment = True, random_seed = 1))

(<torch.utils.data.dataloader.DataLoader object at 0x7f6d554ac8e0>, <torch.utils.data.dataloader.DataLoader object at 0x7f6df4281e50>)


In [22]:
print(train_loader)

In [9]:
test_dataset = torchvision.datasets.GTSRB(
    root='./data', split = 'test', transform=transforms, download=True)

# train loader 
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [53]:
print(len(test_dataset), len(train_dataset), len(val_dataset))

12630 21312 5328


# train model 

In [12]:
def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

In [13]:
def plot_losses(train_losses, valid_losses):
    '''
    Function for plotting training and validation losses
    '''
    
    # temporarily change the style of the plots to seaborn 
    #plt.style.use('seaborn')

    train_losses = np.array(train_losses) 
    valid_losses = np.array(valid_losses)

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='blue', label='Training loss') 
    ax.plot(valid_losses, color='red', label='Validation loss')
    ax.set(title="Loss over epochs", 
            xlabel='Epoch',
            ylabel='Loss') 
    ax.legend()
    fig.show()
    
    # change the plot style to default
    plt.style.use('default')

In [14]:
# train function

def train(train_loader, model, criterion, optimizer, device):
    '''
    Function for the training step of the training loop
    '''

    model.train()
    running_loss = 0
    
    for X, y_true in train_loader:

        optimizer.zero_grad()
        
        X = X.to(device)
        y_true = y_true.to(device)
    
        # Forward pass
        y_hat= model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

        # Backward pass
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

In [15]:
# validation function, without a learning step (backward pass)

def validate(valid_loader, model, criterion, device):
    '''
    Function for the validation step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in valid_loader:
    
        X = X.to(device)
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat= model(X)    # predicted
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
        
    return model, epoch_loss

In [16]:
# training function
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device, print_every=1):
    '''
    Function defining the entire training loop
    '''
    
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    valid_losses = []
    # Train model
    for epoch in range(0, epochs):

        # training
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)

        # validation
        with torch.no_grad():
            model, valid_loss = validate(valid_loader, model, criterion, device)
            valid_losses.append(valid_loss)

        if epoch % print_every == (print_every - 1):
            
            train_acc = get_accuracy(model, train_loader, device=device)
            valid_acc = get_accuracy(model, valid_loader, device=device)
                
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')

    plot_losses(train_losses, valid_losses)
    
    return model, optimizer, (train_losses, valid_losses)

In [17]:
model = resnet50(weights=None)
model.fc = torch.nn.Linear(2048,43)
model.conv1 = torch.nn.Conv2d(3,64,kernel_size=5,stride=1)

model = model.to('cuda')
torch.manual_seed(random_seed)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [18]:
model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device)

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/torch/utils/data/dataset.py", line 295, in __getitem__
    return self.dataset[self.indices[idx]]
  File "/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/torchvision/datasets/folder.py", line 231, in __getitem__
    sample = self.transfo

# adversarial training 

In [ ]:
import numpy as np
import torch

# Load the model
model = MyModel()

# Set the model to evaluation mode
model.eval()

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Set the number of epochs for adversarial training
num_epochs = 10

for epoch in range(num_epochs):
    # Loop over the training data
    for x, y in train_loader:
        # Move the data to the device
        x, y = x.to(device), y.to(device)

        # Create adversarial examples using DeepFool
        x_adv = deepfool(model, x, y, device)
        # Calculate the loss
        loss = loss_fn(model(x_adv), y)
        # Backpropagate the gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Create adversarial examples using Carlini-Wagner
        x_adv = carlini_wagner(model, x, y, device)
        # Calculate the loss
        loss = loss_fn(model(x_adv), y)
        # Backpropagate the gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Create adversarial examples using IFGSM
        x_adv = ifgsm(model, x, y, device)
        # Calculate the loss
        loss = loss_fn(model(x_adv), y)
        # Backpropagate the gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Set the model back to training mode
model.train()
